In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.neighbors import KernelDensity
# from ifeatpro.features import get_feature, get_all_features
import multiprocessing as mp

1. Get training data
2. Create training and validation set
3. Create molecular signature for train protein sequences
4. Get substrate molecular signature file
5. Get substrate molecular signature
6. Encode into a single vector
7. Train KDE model
8. Create random substrate-enzyme pairs
9. Evaluate model

In [2]:
# substrate encoding
subs_encoding_file= "./data/MolecularSignatureKEGGComp/mol_sig_matrix_kegg_comp.csv"
df_subs = pd.read_csv(subs_encoding_file, index_col=0).T

# Sequence Encoding

1. encode them as all ifeature encodings

In [3]:
# help(get_feature)

In [4]:
"""
moran - done
geary - done
nmbroto - done
ctdc - done
ctdt - done
ctdd - done
ctriad - done
ksctriad - done
socnumber - done
qsorder - done
paac - done
apaac - done
"""

# get_feature("./data/training_fasta.fa", "dde", "./data/enz_ifeatures/")

'\nmoran - done\ngeary - done\nnmbroto - done\nctdc - done\nctdt - done\nctdd - done\nctriad - done\nksctriad - done\nsocnumber - done\nqsorder - done\npaac - done\napaac - done\n'

In [5]:
# get_all_features("./data/training_fasta.fa", "./data/enz_ifeatures/")

In [6]:
train_filepath = "./data/training_raw.csv"
train_file = open(train_filepath, "r")

X_raw = []
for i,line in enumerate(train_file):
    name, seq, subs = line.strip().split(",")
    X_raw.append((name, seq, subs))


In [7]:
X_train, X_valid = train_test_split(X_raw, random_state=10)

In [8]:
# filter X_train and X_valid
X_train = [x for x in X_train if x[2] in df_subs.index]
X_valid = [x for x in X_valid if x[2] in df_subs.index]

In [9]:
len(X_train)

51994

In [10]:
len(X_valid)

17415

In [11]:
X_train_name = [x[0] for x in X_train]
X_train_seq = [x[1] for x in X_train]
X_train_subs = [x[2] for x in X_train]

X_valid_name = [x[0] for x in X_valid]
X_valid_seq = [x[1] for x in X_valid]
X_valid_subs = [x[2] for x in X_valid]

In [12]:
X_enc = pd.read_csv("./data/enz_ifeatures/dde.csv", index_col=0)

In [13]:
X_enc_train = X_enc.loc[X_train_name, :]
X_enc_valid = X_enc.loc[X_valid_name, :]

In [14]:
X_enc_train.head()

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,YM,YN,YP,YQ,YR,YS,YT,YV,YW,YY
#,,,,,,,,,,,,,,,,,,,,,
hsa:4976_3.6.5.5,-0.061009,-0.740270,-0.740270,2.048437,-0.043093,-1.541540,-0.740270,1.086292,-0.043093,0.731969,...,0.675085,-0.030455,-0.740270,1.940397,-1.191906,-1.191906,-0.740270,-1.437447,0.675085,0.954971
ggo:101128753_2.5.1.18,0.037596,-0.694715,-0.694715,-0.694715,0.747826,0.037596,0.747826,1.505623,-0.694715,-0.371230,...,-0.347077,-0.490973,-0.694715,-0.490973,-0.851307,0.327158,0.747826,-0.694715,2.535677,-0.490973
ptr:457477_2.5.1.18,0.442999,-0.568497,-0.568497,-0.568497,1.194317,-0.804844,2.957132,0.743469,-0.568497,1.053129,...,-0.284019,-0.401772,2.957132,-0.401772,-0.696639,0.743469,1.194317,-0.568497,-0.284019,2.089882
tac:Ta0619_4.1.2.55,3.387541,-0.787732,0.484469,0.484469,-0.787732,1.586435,-0.787732,-0.965291,-0.787732,0.104845,...,-0.393548,1.241485,1.756671,-0.556711,-0.965291,0.074017,0.484469,-0.787732,-0.393548,1.241485
pmex:H4W19_11290_1.5.8.2,3.525268,-0.392928,2.901909,4.549327,-1.216637,0.609874,2.078200,-1.490873,-1.216637,3.607328,...,-0.607827,0.304445,-0.392928,0.304445,-0.145036,-0.817955,0.430781,0.430781,-0.607827,-0.859829


In [15]:
len(X_enc_train), len(X_enc_valid)

(51994, 17415)

# Substrate Encoding

In [16]:
len(set(df_subs.index).intersection(set(X_train_subs)))

2178

In [17]:
len(set(X_train_subs))

2178

In [18]:
len(set(X_valid_subs)), len(set(df_subs.index).intersection(set(X_valid_subs)))

(1963, 1963)

In [19]:
df_subs.head(2)

,O,cN,cc(n)N,cc(c)n,cnc,ncn,cn(c)C,C[C@H](n)O,COC,C[C@H](C)O,...,[NH]N,[N]=N[NH],CC(=C)[N+],C/C(=C)N,co[nH],c[nH]o,CS(=c)[O-],CS(=c)O,[NH3+]CS,CC([n+])O
C00001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C00002,0.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
len(X_train_subs)

51994

In [21]:
X_train_subs[:5]

['C00001', 'C19586', 'C11088', 'C01286', 'C00565']

In [22]:
X_train_subs_molsig = df_subs.loc[X_train_subs, :]
X_valid_subs_molsig = df_subs.loc[X_valid_subs, :]

In [23]:
X_train_subs_molsig.head(2)

,O,cN,cc(n)N,cc(c)n,cnc,ncn,cn(c)C,C[C@H](n)O,COC,C[C@H](C)O,...,[NH]N,[N]=N[NH],CC(=C)[N+],C/C(=C)N,co[nH],c[nH]o,CS(=c)[O-],CS(=c)O,[NH3+]CS,CC([n+])O
C00001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C19586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# Training and Validation Concatenation of sequences and substrates

X_tr = np.concatenate((X_enc_train.values, X_train_subs_molsig.values), axis=1)
X_va = np.concatenate((X_enc_valid.values, X_valid_subs_molsig.values), axis=1)

In [25]:
X_tr.shape, X_va.shape

((51994, 1839), (17415, 1839))

In [26]:
# creating random enzymes with replacement of size twice that of validation

X_valid_name_rand = np.random.choice(X_valid_name, size=2*len(X_valid_name))
X_valid_subs_rand = np.random.choice(X_valid_subs, size=2*len(X_valid_subs))

In [27]:
# getting rid of any possible positive pairs

positive_valid_set = set(zip(X_valid_name, X_valid_subs))

potential_negative_valid = []

for enz,sub in zip(X_valid_name_rand, X_valid_subs_rand):
    if (enz,sub) in positive_valid_set:
        print((enz,sub))
    else:
        potential_negative_valid.append((enz,sub))

('micc:AUP74_02544_1.12.1.5', 'C00003')
('azi:AzCIB_4622_1.3.7.8', 'C00001')
('psk:U771_11545_1.1.3.47', 'C00001')
('pps:100973371_2.7.11.25', 'C00002')
('nle:100596642_2.4.1.17', 'C05302')
('eba:ebA5287_1.3.7.8', 'C00002')
('qsu:111995922_3.6.1.10', 'C00001')
('pon:100171657_3.3.2.6', 'C00001')
('nle:100596640_2.7.11.14', 'C00002')
('pon:100441255_3.1.3.16', 'C00001')
('ppr:PBPRB0650_1.7.7.2', 'C00001')
('pon:100171524_2.6.1.44', 'C00026')
('pon:100440274_3.5.2.17', 'C00001')
('ptr:468438_1.14.11.68', 'C00007')
('pps:100970340_3.4.17.23', 'C00001')
('tbr:Tb927.8.2210_1.5.1.33', 'C00006')
('dji:CH75_12785_3.1.2.29', 'C00001')
('crb:17880559_2.7.1.71', 'C00002')
('hsa:55031_3.4.19.12', 'C00001')
('epa:110238942_1.5.3.14', 'C00001')
('pam:PANA_3819_1.14.11.75', 'C00026')
('mthi:C7M52_04028_1.21.4.4', 'C00001')
('kpe:KPK_1712_1.18.6.1', 'C00002')
('hsa:25862_3.4.19.12', 'C00001')
('ptr:743223_6.3.4.5', 'C00002')
('spu:584984_1.10.3.2', 'C00007')
('aly:9314456_2.1.2.11', 'C00001')
('hsa:57

In [28]:
len(potential_negative_valid)

34182

In [29]:
X_valid_name_neg = [pn[0] for pn in potential_negative_valid]
X_valid_subs_neg = [pn[1] for pn in potential_negative_valid]

X_enc_valid_neg = X_enc.loc[X_valid_name_neg, :]
X_valid_subs_molsig_neg = df_subs.loc[X_valid_subs_neg, :]

X_va_neg = np.concatenate((X_enc_valid_neg.values, X_valid_subs_molsig_neg.values), axis=1)

In [30]:
class EnzymeRanker:
    def __init__(self, X_tr_pos, kern="gaussian", bw=0.35):
        """
        X_tr_pos: An array of concatenated enzyme and substrate feature vectors required for model training
        kernel: kde model kernel
        bw: bandwidth of the kde model
        """
        self.kern =  kern
        self.bw = bw
        self.model = self._get_model()
        self.model.fit(X_tr_pos)
    
    def _get_model(self):
        return KernelDensity(kernel=self.kern, bandwidth=self.bw)
    
    def get_pos_acc(self, X_va_pos):
        scores_valid = self.model.score_samples(X_va_pos)
        correct_frac = len(np.where(scores_valid>0)[0])/len(scores_valid)
        return correct_frac, scores_valid
    
    def get_neg_acc(self, X_va_neg):
        scores_valid = self.model.score_samples(X_va_neg)
        correct_frac = len(np.where(scores_valid<0)[0])/len(scores_valid)
        return correct_frac, scores_valid
    

In [31]:
def bandwidth_hpopt(bw):
    kde = EnzymeRanker(X_tr, bw=bw)
    acc_pos, _ = kde.get_pos_acc(X_va)
    acc_neg, _ = kde.get_neg_acc(X_va_neg)
    return bw, acc_pos, acc_neg

In [32]:
mp.cpu_count()

24

In [33]:
def multi_func(bws):
    pool = mp.Pool(mp.cpu_count())
    info = pool.map(bandwidth_hpopt, bws)
    return info

In [44]:
bws_ = np.logspace(-0.41, -0.405, 50)

In [45]:
%%time
scores = multi_func(bws_)

CPU times: user 1.02 s, sys: 415 ms, total: 1.44 s
Wall time: 5h 35min 43s


In [46]:
scores

[(0.3890451449942806, 0.7444731553258686, 0.5619916915335557),
 (0.3891365648716548, 0.7594028136663795, 0.5219121174887368),
 (0.3892280062313534, 0.7912718920470858, 0.4293487800596805),
 (0.3893194690784243, 0.7552110249784668, 0.5193376630975367),
 (0.3894109534179167, 0.7475739305196669, 0.553419928617401),
 (0.38950245925488114, 0.7405684754521964, 0.5393774501199462),
 (0.3895939865943691, 0.7519954062589721, 0.5262126265285823),
 (0.3896855354414334, 0.7480907263853, 0.524983909660055),
 (0.389777105801128, 0.7626758541487224, 0.4915160025744544),
 (0.38986869767850807, 0.7374677002583979, 0.5390556433210462),
 (0.3899603110786299, 0.7101923629055412, 0.5965420396700017),
 (0.390051946006551, 0.7099052540913006, 0.5902229243461471),
 (0.3901436024673302, 0.7444157335630204, 0.5322684453806097),
 (0.3902352804660273, 0.7489520528280218, 0.5087180387338365),
 (0.3903269800077034, 0.7541774332472007, 0.5158562986367093),
 (0.39041870109742083, 0.7335055986218777, 0.541834883857000

In [ ]:
%%time
kde = EnzymeRanker(X_tr, bw=0.39290333)

In [55]:
kde.model.score(X_tr[:40])

1241.42743560511

In [56]:
%%time
acc_pos, scores_pos = kde.get_pos_acc(X_va)

CPU times: user 25min 44s, sys: 1.3 s, total: 25min 46s
Wall time: 25min 43s


In [57]:
%%time
acc_neg, scores_neg = kde.get_neg_acc(X_va_neg)

CPU times: user 50min 49s, sys: 4.95 s, total: 50min 54s
Wall time: 50min 45s


In [58]:
print(acc_pos, acc_neg)

0.7444731553258686 0.5399683766690091


In [59]:
return_statistics(scores_neg)

(-466.0266034021124,
 -18.478406064283842,
 974.5862666819726,
 -8900.24583571762,
 33.396207987457814)

In [60]:
return_statistics(scores_pos)

(-86.70315477234887,
 7.856755167973606,
 434.42926400412193,
 -7584.30051998923,
 33.396207987457814)

In [34]:
with open("positive_pair_scores.csv", "w") as f:
    for enz,subs,score in zip(X_valid_name, X_valid_subs, scores_pos):
        f.write(f"{enz},{subs},{score}\n")

In [35]:
with open("negative_pair_scores.csv", "w") as f:
    for enz,subs,score in zip(X_valid_name_neg, X_valid_subs_neg, scores_neg):
        f.write(f"{enz},{subs},{score}\n")

# Validation Score statistics

In [45]:
def return_statistics(scores):
    return np.mean(scores), np.median(scores), np.std(scores), np.min(scores), np.max(scores)

In [46]:
return_statistics(scores_neg)

(-7877.43221164863,
 1185.1128031780004,
 17520.357731976044,
 -133308.01546518086,
 2536.232089203827)

In [37]:
return_statistics(scores_valid)

(70.60164939712396,
 205.34652975492108,
 575.5400247500572,
 -9225.360232914447,
 232.40099014084677)

In [38]:
return_statistics(scores_valid_neg)

(-456.35403101636416,
 148.896534902722,
 1279.0112968122646,
 -9736.00232334976,
 232.40099014084677)

## Scores with very low activity predicted (<1)

In [39]:
len(np.where(scores_valid<1)[0]), len(np.where(scores_valid_neg<1)[0])

(1732, 11871)

# Model Evaluation

# Tanimoto index comparison

Main idea: When compared to a given substrate, if a similar substrate interacts with an enzyme, then the given substrate will also interact with that enzyme. 

Given an enzyme-substrate pair:

- Look at the substrate name
- Select a threshold
- Find substrates which are known to be similar based on that threshold
- Find enzymes known to react with that substrate
- Is the given enzyme present in that list? 
    - Yes: Enzyme reacts with that substrate.
    - No: It does not react with that substrate.

In [40]:
def get_tanimoto_index(sub1, sub2):
    """
    Tanimoto index calculator: requires substrate 1 and 2 features/signatures
    """
    bool_sub1 = set(np.where(sub1>=1)[0])
    bool_sub2 = set(np.where(sub2>=1)[0])
    Num_feat_sub1 = len(bool_sub1)
    Num_feat_sub2 = len(bool_sub2)
    Num_common = len(bool_sub1.intersection(bool_sub2))
    return Num_common/(Num_feat_sub1 + Num_feat_sub2 - Num_common)

In [41]:
# Multiple thresholds for the Tanimoto model
thres = [0.1, 0.33, 0.5, 0.66, 0.9]



In [42]:
# sort the database by substrate name and then enzyme name
X_tani_database = sorted(list(zip(X_train_name,  X_train_subs)), key=lambda x: (x[1], x[0]))

In [43]:
import itertools

In [44]:
tani_iterator = itertools.groupby(X_tani_database, lambda x : x[1])
tani_data_dir = dict()
  
for key, group in tani_iterator:
    tani_data_dir[key] = [g[0] for g in group]


In [45]:
df_subs.head(2)

,O,cN,cc(n)N,cc(c)n,cnc,ncn,cn(c)C,C[C@H](n)O,COC,C[C@H](C)O,...,[NH]N,[N]=N[NH],CC(=C)[N+],C/C(=C)N,co[nH],c[nH]o,CS(=c)[O-],CS(=c)O,[NH3+]CS,CC([n+])O
C00001,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C00002,0.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df_subs.loc["C00001", :].values

array([1., 0., 0., ..., 0., 0., 0.])

In [54]:
%%time
correct = 0
thresh = 0.33
k=5000
i=0

# iterate through validation data pairs
for enz,subs in zip(X_valid_name[:k], X_valid_subs[:k]):
    # get given substrate feature
    sub_feat = df_subs.loc[subs,:].values
    i += 1
    progress=i/k 
    if progress in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        print("*",end="")
    
    # iterate through all other substrates in df_subs
    for nat_sub in df_subs.index:
        if nat_sub != subs:
            nat_sub_feat = df_subs.loc[nat_sub,:].values
            # get tanimoto index between native and given substrate
            tani_idx = get_tanimoto_index(sub_feat, nat_sub_feat)
            # if tani is greater than threshold
            if tani_idx >= thresh:
                if nat_sub in tani_data_dir:
                    reacting_enz_list = tani_data_dir[nat_sub]
                    if enz in reacting_enz_list:
                        correct += 1
                        break

*********CPU times: user 2h 13min 12s, sys: 3.85 s, total: 2h 13min 16s
Wall time: 2h 13min 15s


In [55]:
correct/k

0.3974

In [56]:
len(np.where(scores_valid[:k]>1)[0])/k

0.9062

In [57]:
%%time
incorrect = 0
thresh = 0.33
k=5000
i=0

# iterate through validation data pairs
for enz,subs in zip(X_valid_name_neg[:k], X_valid_subs_neg[:k]):
    # get given substrate feature
    sub_feat = df_subs.loc[subs,:].values
    i += 1
    progress=i/k 
    if progress in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        print("*",end="")
    
    # iterate through all other substrates in df_subs
    for nat_sub in df_subs.index:
        if nat_sub != subs:
            nat_sub_feat = df_subs.loc[nat_sub,:].values
            # get tanimoto index between native and given substrate
            tani_idx = get_tanimoto_index(sub_feat, nat_sub_feat)
            # if tani is greater than threshold
            if tani_idx >= thresh:
                if nat_sub in tani_data_dir:
                    reacting_enz_list = tani_data_dir[nat_sub]
                    if enz in reacting_enz_list:
                        incorrect += 1
                        break

*********CPU times: user 2h 52min 55s, sys: 3.44 s, total: 2h 52min 58s
Wall time: 2h 52min 57s


In [58]:
(k-incorrect)/k

0.8884

In [59]:
len(np.where(scores_valid_neg[:k]<1)[0])/k

0.3486